### Files pending meshing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

def get_first_unprocessed_png(png_folder, review_folder):
    png_files = [file for file in os.listdir(png_folder) if file.endswith('.png')]
    png_names = [os.path.splitext(file)[0] for file in png_files]

    # Check in the results folder if there are folders with the names of the .png files
    names_to_remove = []
    for name in png_names:
        folder_path = os.path.join(review_folder, name)
        if os.path.isdir(folder_path):
            names_to_remove.append(name)

    # Remove the names that have corresponding folders from the list
    final_names = [name for name in png_names if name not in names_to_remove]

    #return f"{final_names[0]}.png" if final_names else None
    first_unprocessed = f"{final_names[0]}.png" if final_names else None
    return [first_unprocessed, len(png_files), len(png_files)-len(final_names)]

In [ ]:
# Paths: files and results

png_folder = ''
review_folder = ''

unprocessed_png = get_first_unprocessed_png(png_folder, review_folder)
print("first_unprocessed, total, done")
print(unprocessed_png)

first_unprocessed, total, done
['B0029_rembg.png', 106, 31]


### Clone repository

In [ ]:
!git clone https://github.com/aletrujim/body2vec_mesh.git

Cloning into 'body2vec_mesh'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 118 (delta 14), reused 112 (delta 12), pack-reused 3 (from 1)
Receiving objects: 100% (118/118), 72.12 MiB | 13.21 MiB/s, done.
Resolving deltas: 100% (14/14), done.


### Configure input / output data

In [ ]:
# input
file_name = unprocessed_png[0] #first_unprocessed ".png"
print("file_name ", file_name)

image_path_ori = "/%s" % file_name

file_name  B0029_rembg.png


In [ ]:
import shutil
import os

# # input path
shutil.copy(image_path_ori, "")
image_path = '/%s' % file_name
image_dir = os.path.dirname(image_path)

# output paths
obj_path = '/result_%s.obj' % file_name
out_img_path = '/result_%s.png' % file_name

### Preprocess: human-pose

In [ ]:
from os import chdir as cd

cd('/content/body2vec_mesh/human-pose')

In [ ]:
import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo

#print(torch.__version__)

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)

            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int_)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int_)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

In [ ]:
net = PoseEstimationWithMobileNet()
checkpoint = torch.load('/content/body2vec_mesh/checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)

get_rect(net.cuda(), [image_path], 512)

### Pretrained Model: pifuhd

In [ ]:
cd('/content/body2vec_mesh/pifuhd/')

In [ ]:
!sh ./scripts/download_trained_model.sh

+ mkdir -p checkpoints
+ cd checkpoints
+ wget https://dl.fbaipublicfiles.com/pifuhd/checkpoints/pifuhd.pt pifuhd.pt
--2024-08-21 18:22:44--  https://dl.fbaipublicfiles.com/pifuhd/checkpoints/pifuhd.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.18.103, 13.35.18.102, 13.35.18.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.18.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1548375177 (1.4G) [application/octet-stream]
Saving to: ‘pifuhd.pt’

pifuhd.pt           100%[===================>]   1.44G   135MB/s    in 14s     

2024-08-21 18:22:58 (109 MB/s) - ‘pifuhd.pt’ saved [1548375177/1548375177]

--2024-08-21 18:22:58--  http://pifuhd.pt/
Resolving pifuhd.pt (pifuhd.pt)... failed: Name or service not known.
wget: unable to resolve host address ‘pifuhd.pt’
FINISHED --2024-08-21 18:22:58--
Total wall clock time: 14s
Downloaded: 1 files, 1.4G in 14s (109 MB/s)


### Run: get new mesh


In [ ]:
# 256 is the maximum resolution that can fit into Google Colab
!python -m apps.simple_test -r 256 --use_rect -i $image_dir

Resuming from  ./checkpoints/pifuhd.pt
test data size:  1
initialize network with normal
initialize network with normal
generate mesh (test) ...
  0% 0/1 [00:00<?, ?it/s]./results/pifuhd_final/recon/result_B0029_rembg_256.obj
100% 1/1 [00:05<00:00,  5.96s/it]


### Save results

In [ ]:
results_path = ''
folder_name = file_name.split(".")[0]

save_results_path = '/%s' % folder_name

shutil.copytree(results_path, save_results_path)

'/content/drive/My Drive/body2vec/Bahia/dataset/fotos/3D_results_bahia/B0029_rembg'

### Download

In [ ]:
#from google.colab import files
#files.download('')